In [4]:
import cryptocompare
import pandas as pd
import json
from datetime import datetime, timedelta
import blankly
import matplotlib.pyplot as plt

# mount personal drive

BACKTEST_DATA_LOCATION ='/content/gdrive/My Drive/Colab Notebooks/Algo Trading/data/btc_sentiments.json'
START_DATE = '2017-08-01'
END_DATE = '2024-08-01'

In [3]:
def download_btc_price_data(
        start_date, 
        end_date, 
        output_filename='btc_price_data', 
        verbose=False
        ):

    start_date = datetime.fromisoformat(start_date).date()
    end_date = datetime.fromisoformat(end_date).date()

    # Make sure start date is before end, if not then switch them
    if start_date > end_date:
        tmp = start_date
        start_date = end_date
        end_date = tmp

    if start_date == end_date:
        raise ValueError("start_date cannot be the same as the end date")

    start_time_epoch = datetime.combine(start_date, datetime.min.time()).timestamp()
    end_time_epoch = datetime.combine(end_date, datetime.min.time()).timestamp()

    btc_hist_df = None
    current_date = start_date
    total_days_needed = (end_date - start_date).days

    if verbose:
        print(f'Collecting {total_days_needed} days of historical BTC data..')

    while total_days_needed > 0:
    # max 1 year allowed per request on free tier, send multiple requests instead
        limit_days = min(total_days_needed, 365)
        to_ts = datetime.combine(current_date + timedelta(days=limit_days - 1), datetime.min.time()).timestamp()  # End of current chunk
        partial_btc_data = cryptocompare.get_historical_price_day(
            # grab historical data from cryptocompare library
            coin='BTC',
            currency='USD',
            limit=limit_days,
            toTs=int(to_ts),
        )
        if verbose:
            print(f'There are {total_days_needed} days left to collect!')

        if btc_hist_df is None:
            btc_hist_df = pd.DataFrame(partial_btc_data)
        else:
            btc_hist_df = pd.concat([btc_hist_df, pd.DataFrame(partial_btc_data)], axis=0)

        # increment current_date and deincrement total_days_needed
        current_date += timedelta(days=limit_days)
        total_days_needed -= limit_days

    btc_hist_df = btc_hist_df[['time', 'low', 'high', 'open', 'close', 'volumefrom']]
    btc_hist_df.columns = ['time', 'low', 'high', 'open', 'close', 'volume']
    btc_hist_df = btc_hist_df.drop_duplicates(subset=['time'], keep='first')
    btc_hist_df.to_csv(f'{output_filename}.csv', index=False)

    if verbose:
        print(f'min: {btc_hist_df["time"].min()}')
        print(f'max: {btc_hist_df["time"].max()}')
        print(f'start time epoch: {start_time_epoch}')
        print(f'end time epoch: {end_time_epoch}') 

There are 2557 days left to collect!
There are 2192 days left to collect!
There are 1827 days left to collect!
There are 1462 days left to collect!
There are 1097 days left to collect!
There are 732 days left to collect!
There are 367 days left to collect!
There are 2 days left to collect!
min: 1501459200
max: 1722384000
start time epoch: 1501567200.0
end time epoch: 1722492000.0


In [ ]:
if test is True:
